# Use an Agent as a tool for another agent.

Multi-Agent systems are complicated! Enter the [A2A](https://github.com/google-a2a/A2A) protocol by Google: this protocol allows for simpler communication between agents, and easily enables the use of an agent as a tool for another agent. In this tutorial we'll show how you can create two agents with any-agent and have one agent be provided as a tool for the other agent.

This tutorial assumes basic familiarity with any-agent: if you haven't used any-agent before you may also find the [Creating your first agent](./../your_first_agent) cookbook to be useful.


Note: because this tutorial relies upon advanced stdio/stderr communication using the MCP Server, it cannot be run on Google Colab.


## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [ ]:
%pip install 'any-agent[a2a]'

import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
from getpass import getpass

# This notebook communicates with OpenAI GPT models using the OpenAI API.
if "OPENAI_API_KEY" not in os.environ:
    print("OPENAI_API_KEY not found in environment!")
    api_key = getpass("Please enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = api_key
    print("OPENAI_API_KEY set for this session!")
else:
    print("OPENAI_API_KEY found in environment.")

OPENAI_API_KEY found in environment.


## Configure the first agent and serve it over A2A

Let's give our first "helper" agent the very simple capability to access the current time through a Model Context Protocol (MCP) server. For this demo, we'll use the async method `agent.serve_async` so that we can easily run both this agent and the second agent from inside the notebook. 

In [ ]:
from any_agent import AgentConfig, AnyAgent
from any_agent.config import MCPStdio, ServingConfig, TracingConfig

# This MCP Tool relies upon uvx https://docs.astral.sh/uv/getting-started/installation/
time_tool = MCPStdio(
    command="uvx",
    args=["mcp-server-time", "--local-timezone=America/New_York"],
    tools=[
        "get_current_time",
    ],
)

time = AnyAgent.create(
    "openai",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(
        model_id="gpt-4.1-nano",
        description="I'm an agent to help with getting the time",
        tools=[time_tool],
    ),
    tracing=TracingConfig(console=False),
)
(time_task, time_server) = await time.serve_async(
    ServingConfig(port=5000, endpoint="/time")
)

### Configure and use the Second Agent 

Now that the first agent is serving over A2A, our second agent can be given access to use it just like a tool! In order to do this you use the `a2a_tool` function which retrieves the info about the agent and allows you to call it. 

In [4]:
from any_agent import AgentConfig, AnyAgent
from any_agent.config import TracingConfig
from any_agent.tools import a2a_tool

config = AgentConfig(
    model_id="gpt-4.1-mini",
    instructions="Use the available tools to obtain additional information to answer the query.",
    description="The orchestrator that can use other agents via tools using the A2A protocol.",
    tools=[
        await a2a_tool(
            "http://localhost:5000/time",
            http_kwargs={
                "timeout": 5
            },  # This gives the time agent up to 5 seconds to respond to each request
        )
    ],
)

agent = await AnyAgent.create_async(
    agent_framework="tinyagent",
    agent_config=config,
    tracing=TracingConfig(console=True),
)

agent_trace = await agent.run_async("What date and time is it right now?")

print(agent_trace.final_output)

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "system",                                                                                       │ │
│ │     "content": "Use the available tools to obtain additional information to answer the query."              │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "What date and time is it right now?"                                                        │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "call_any_agent",                                                                          │ │
│ │     "tool.args": "{\"query\":\"current date and time\"}"                                                    │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 146,                                                                                      │ │
│ │   "output_tokens": 18,                                                                                      │ │
│ │   "input_cost": 5.8399999999999997e-05,                                                                     │ │
│ │   "output_cost": 2.88e-05                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "current date and time"                                                                      │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_current_time",                                                                        │ │
│ │     "tool.args": "{\"timezone\":\"America/New_York\"}"                                                      │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 214,                                                                                      │ │
│ │   "output_tokens": 18,                                                                                      │ │
│ │   "input_cost": 2.14e-05,                                                                                   │ │
│ │   "output_cost": 7.2e-06                                                                                    │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "current date and time"                                                                      │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_current_time",                                                                        │ │
│ │     "tool.args": "{\"timezone\":\"America/New_York\"}"                                                      │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 214,                                                                                      │ │
│ │   "output_tokens": 18,                                                                                      │ │
│ │   "input_cost": 2.14e-05,                                                                                   │ │
│ │   "output_cost": 7.2e-06                                                                                    │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── EXECUTE_TOOL: get_current_time ─────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "timezone": "America/New_York"                                                                            │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "type": "text",                                                                                           │ │
│ │   "text": "{\n  \"timezone\": \"America/New_York\",\n  \"datetime\": \"2025-05-29T20:05:53-04:00\",\n  \"is │ │
│ │   "annotations": null                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ The current date and time in New York is May 29, 2025, 20:05:53.                                            │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 297,                                                                                      │ │
│ │   "output_tokens": 25,                                                                                      │ │
│ │   "input_cost": 2.97e-05,                                                                                   │ │
│ │   "output_cost": 9.999999999999999e-06                                                                      │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ The current date and time in New York is May 29, 2025, 20:05:53.                                            │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 297,                                                                                      │ │
│ │   "output_tokens": 25,                                                                                      │ │
│ │   "input_cost": 2.97e-05,                                                                                   │ │
│ │   "output_cost": 9.999999999999999e-06                                                                      │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────── EXECUTE_TOOL: call_any_agent ──────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "current date and time"                                                                          │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "content": [                                                                                              │ │
│ │     {                                                                                                       │ │
│ │       "text": "{\"id\":\"cc02575859f24d19a02f42d12a781de6\",\"jsonrpc\":\"2.0\",\"result\":{\"contextId\":n │ │
│ │     }                                                                                                       │ │
│ │   ]                                                                                                         │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ The current date and time in New York is May 29, 2025, 20:05:53.                                            │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 298,                                                                                      │ │
│ │   "output_tokens": 25,                                                                                      │ │
│ │   "input_cost": 0.0001192,                                                                                  │ │
│ │   "output_cost": 3.9999999999999996e-05                                                                     │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-mini ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ The current date and time right now is May 29, 2025, 20:05:53 (New York time).                              │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 326,                                                                                      │ │
│ │   "output_tokens": 28,                                                                                      │ │
│ │   "input_cost": 0.0001304,                                                                                  │ │
│ │   "output_cost": 4.48e-05                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The current date and time right now is May 29, 2025, 20:05:53 (New York time).


In [5]:
import asyncio

time_server.should_exit = True
while True:
    if time_task.done():
        print("Agent server task has completed!")
        break
    else:
        print("Waiting for server to exit")
        await asyncio.sleep(1)
agent.exit()

Waiting for server to exit
Agent server task has completed!
